# FHOPS Debug Workbench
Interactive notebook to load a scenario, build the MIP model, run the solver, evaluate KPIs, and debug issues.

**Tips**
- Run this from the FHOPS repo root (same folder as `pyproject.toml`).
- In CodeServer/Jupyter with the debugger enabled, uncaught exceptions will pause execution.


In [1]:
# --- Environment sanity check ---
import sys, os, platform, importlib, pathlib
from pprint import pprint

PROJECT_ROOT = pathlib.Path.cwd()
assert (PROJECT_ROOT / "src" / "fhops").exists(), "Run this notebook from FHOPS repo root."

if str(PROJECT_ROOT / "src") not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT / "src"))
import fhops
print("Python:", sys.version)
print("Platform:", platform.platform())
print("FHOPS version:", fhops.__version__)
print("FHOPS package path:", fhops.__file__)


Python: 3.12.3 (main, Jun 18 2025, 17:59:45) [GCC 13.3.0]
Platform: Linux-6.14.0-27-generic-x86_64-with-glibc2.39
FHOPS version: 0.1.0
FHOPS package path: /home/gep/projects/fhops/src/fhops/__init__.py


## 1) Load scenario & inspect parsed data

In [2]:
from fhops.data.loaders import load_scenario
from fhops.core.types import Problem

SCENARIO_PATH = (pathlib.Path.cwd() / "examples" / "minitoy" / "scenario.yaml")
sc = load_scenario(SCENARIO_PATH)
pb = Problem.from_scenario(sc)

print("Scenario:", sc.name)
print("Days:", len(pb.days), "Blocks:", len(sc.blocks), "Machines:", len(sc.machines), "Landings:", len(sc.landings))
print("First block:", sc.blocks[0].model_dump())
print("First machine:", sc.machines[0].model_dump())


Scenario: FHOPS MiniToy
Days: 14 Blocks: 3 Machines: 2 Landings: 2
First block: {'id': 'B1', 'landing_id': 'L1', 'work_required': 20.0, 'earliest_start': 1, 'latest_finish': 14}
First machine: {'id': 'H1', 'crew': 'C1', 'daily_hours': 10.0, 'operating_cost': 1000.0}


## 2) Inspect model builder source

In [3]:
import inspect
import fhops.model.pyomo_builder as pbuilder
print("pyomo_builder file:", pbuilder.__file__)
src = inspect.getsource(pbuilder.build_model)
print(src)


pyomo_builder file: /home/gep/projects/fhops/src/fhops/model/pyomo_builder.py
def build_model(pb: Problem) -> pyo.ConcreteModel:
    sc = pb.scenario

    M = [m.id for m in sc.machines]
    B = [b.id for b in sc.blocks]
    D = list(pb.days)

    rate = {(r.machine_id, r.block_id): r.rate for r in sc.production_rates}
    work_required = {b.id: b.work_required for b in sc.blocks}
    landing_of = {b.id: b.landing_id for b in sc.blocks}
    landing_capacity = {l.id: l.daily_capacity for l in sc.landings}

    # Availability: 1 if machine available on day
    avail = {(c.machine_id, c.day): int(c.available) for c in sc.calendar}

    # Windows: blocks may only be worked between [es, lf]
    windows = {b_id: sc.window_for(b_id) for b_id in sc.block_ids()}

    m = pyo.ConcreteModel()
    m.M = pyo.Set(initialize=M)
    m.B = pyo.Set(initialize=B)
    m.D = pyo.Set(initialize=D)

    def within_window(b, d):
        es, lf = windows[b]
        return 1 if (d >= es and d <= lf) else 0

   

### Optional Hotfix: Window mapping bug
If you see `windows = {b.id: sc.window_for(b.id) for b in sc.block_ids()}`, patch it:
`windows = {b_id: sc.window_for(b_id) for b_id in sc.block_ids()}`
Set `APPLY_FIX = True` to patch automatically.

In [4]:
from pathlib import Path
import importlib, fhops.model.pyomo_builder as pbuilder_mod

APPLY_FIX = True  # <- set True and re-run to patch

file_path = Path(pbuilder_mod.__file__)
text = file_path.read_text(encoding="utf-8")
buggy = "windows = {b.id: sc.window_for(b.id) for b in sc.block_ids()}"
fixed = "windows = {b_id: sc.window_for(b_id) for b_id in sc.block_ids()}"
if buggy in text:
    print("Buggy line detected.")
    if APPLY_FIX:
        file_path.write_text(text.replace(buggy, fixed), encoding="utf-8")
        importlib.reload(pbuilder_mod)
        print("Patched and reloaded.")
    else:
        print("Set APPLY_FIX=True to patch automatically.")
else:
    print("No buggy line found.")


No buggy line found.


## 3) Build the Pyomo model

In [5]:
from fhops.model.pyomo_builder import build_model
import pyomo.environ as pyo
m = build_model(pb)  # debugger stops here on error
print("Model built.")
print(f"|M|={len(m.M)}, |B|={len(m.B)}, |D|={len(m.D)}")
n_vars = sum(1 for _ in m.component_data_objects(pyo.Var))
n_cons = sum(1 for _ in m.component_data_objects(pyo.Constraint))
print("Variables:", n_vars, "Constraints:", n_cons)


Model built.
|M|=2, |B|=3, |D|=14
Variables: 168 Constraints: 143


## 4) Solve with HiGHS (exact)

In [6]:
from fhops.solve.highs_mip import solve_mip
res_mip = solve_mip(pb, time_limit=30)
mip_df = res_mip["assignments"]
print("MIP objective:", res_mip["objective"])
mip_df.head()


MIP objective: 54.0


,machine_id,block_id,day,assigned,production
0,H1,B1,1,1,1.0
14,H2,B2,1,1,3.0
5,H1,B2,2,1,1.0
19,H2,B3,3,1,2.0
1,H1,B1,4,1,3.5


## 5) Solve with Simulated Annealing (heuristic)

In [10]:
from fhops.solve.heuristics.sa import solve_sa
res_sa = solve_sa(pb, iters=1000000, seed=1)
sa_df = res_sa["assignments"]
print("SA objective:", res_sa["objective"])
sa_df.head()


SA objective: 54.0


,machine_id,block_id,day,assigned
6,H2,B2,1,1
0,H1,B1,2,1
7,H2,B2,2,1
8,H2,B2,3,1
9,H2,B3,4,1


## 6) Evaluate KPIs

In [11]:
from fhops.eval.kpis import compute_kpis
k_mip = compute_kpis(pb, mip_df)
k_sa  = compute_kpis(pb, sa_df)
print("MIP KPIs:", k_mip)
print("SA  KPIs:", k_sa)


MIP KPIs: {'total_production': 54.0, 'completed_blocks': 3}
SA  KPIs: {'total_production': 54.0, 'completed_blocks': 3}


## 7) Save schedules to CSV (optional)

In [12]:
out_dir = pathlib.Path.cwd() / "examples" / "minitoy" / "out"
out_dir.mkdir(parents=True, exist_ok=True)
(out_dir / "mip_solution.csv").write_text(mip_df.to_csv(index=False))
(out_dir / "sa_solution.csv").write_text(sa_df.to_csv(index=False))
print("Saved:", out_dir)


Saved: /home/gep/projects/fhops/examples/minitoy/out
